In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from utils import get_knn_objects, get_sample_1k_objects
from enums import DatasetDirs

In [3]:
from scipy.optimize import linear_sum_assignment
from sklearn.model_selection import train_test_split

from imports import logging

from pathlib import Path
from timeit import default_timer as timer

from utils import get_knn_objects, get_sample_1k_objects
from enums import DatasetDirs, DEFAULT_DESCRIPTORS, KNN_OBJECTS

from LMI import LMI
from knn_search import evaluate_knn_per_query

31-03-21 16:28 INFO: Loading faiss with AVX2 support.
31-03-21 16:28 INFO: Loading faiss.


In [4]:
li = LMI(DatasetDirs.MOCAP.value)
df = li.get_dataset(normalize = True)
li.knn_gts_file = '/storage/brno6/home/tslaninakova/learned-indexes/MIndex1M-mocap/knn_gt.json'

Loading Mocap dataset.
Loading labels


/auto/brno6/home/wollf/learned-indexes/learned-indexes/dataset_parsing.py:152: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_i = pd.concat([df_i, df_])


Loading descriptors


31-03-21 16:38 INFO: Loaded dataset of shape: (354902, 4099)


In [5]:
df_result = li.train(df, {"GMM": [{'comp': 71, 'cov_type': 'spherical', 'max_iter': 1, 'init_params': 'random'},
                                  {'comp': 71, 'cov_type': 'spherical', 'max_iter': 1, 'init_params': 'random'}]},
                     should_erase = True)
df_result.shape

31-03-21 16:38 INFO: Training GMM model with values of shape (354902, 4096): n. of clusters=71 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:41 INFO: Training level 1
31-03-21 16:42 INFO: Training GMM model with values of shape (444, 4096): n. of clusters=71 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (50, 4096): n. of clusters=71 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 WARNING: Reducing the number of components from 71 to 25 since the number of                           training samples (50) is less than 71
31-03-21 16:42 INFO: Training GMM model with values of shape (7586, 4096): n. of clusters=25 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (920, 4096): n. of clusters=25 | covariance type=spherical | max iteration

31-03-21 16:42 INFO: Training GMM model with values of shape (79, 4096): n. of clusters=4 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (4621, 4096): n. of clusters=4 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (1748, 4096): n. of clusters=4 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (392, 4096): n. of clusters=4 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (10865, 4096): n. of clusters=4 | covariance type=spherical | max iterations=1 | initial parameters=random
31-03-21 16:42 INFO: Training GMM model with values of shape (311, 4096): n. of clusters=4 | covariance type=spherical | max iterations=1 | initial parameters=random
3

(354902, 4101)

In [6]:
final_results = {}

object_checkpoints = [500, 1000, 3000, 5000, 10000, 50000, 100000, 200000, 300000]
checkpoint_count = len(object_checkpoints) 

knns = li.load_knns()
knns2 = {}

for k, v in knns.items():
    tmp = {}
    for k1, v1 in v.items():
        tmp[k1.replace("-", "_").replace("_", "")] = v1
    knns2[k.replace("-", "_").replace("_", "")] = tmp

queries = get_sample_1k_objects(df_result, "/storage/brno6/home/tslaninakova/learned-indexes/datasets/mocap-queries.data")
query_count = len(queries)

model_times = [0] * checkpoint_count
model_recall = [0] * checkpoint_count

In [7]:
df_new = df_result.copy()
df_new["object_id"] = df_new["object_id"].str.replace("-", "_").apply(int)
df_new.head()["object_id"]

0    3169316915814681581468
1        313731377686276862
2      34173417228693228693
3      31953195380462380462
4      32873287189139189139
Name: object_id, dtype: object

In [13]:
search_result = li.search(df_new, queries.iloc[500]["object_id"], stop_cond_objects = object_checkpoints, debug = False)

In [14]:
search_result

{'id': 3404340419711391971139,
 'time_checkpoints': [0.029683589935302734,
  0.030463457107543945,
  0.030480384826660156,
  0.03098583221435547,
  0.030998945236206055,
  0.05174136161804199,
  0.05430316925048828,
  0.08041214942932129,
  0.08194446563720703],
 'popped_nodes_checkpoints': [['M.1.43', 'C.1.43.1'],
  ['M.1.43', 'C.1.43.1', 'M.1.10'],
  ['M.1.43', 'C.1.43.1', 'M.1.10', 'C.1.10.2'],
  ['M.1.43', 'C.1.43.1', 'M.1.10', 'C.1.10.2', 'M.1.63'],
  ['M.1.43', 'C.1.43.1', 'M.1.10', 'C.1.10.2', 'M.1.63', 'C.1.63.1'],
  ['M.1.43',
   'C.1.43.1',
   'M.1.10',
   'C.1.10.2',
   'M.1.63',
   'C.1.63.1',
   'M.1.34',
   'C.1.34.1',
   'M.1.25',
   'C.1.25.8',
   'C.1.25.3',
   'M.1.40',
   'C.1.40.3',
   'M.1.67',
   'C.1.67.1',
   'M.1.37',
   'C.1.37.2',
   'M.1.1',
   'C.1.1.2',
   'M.1.35',
   'C.1.35.3',
   'M.1.4',
   'C.1.4.6',
   'M.1.41',
   'C.1.41.3',
   'M.1.22',
   'C.1.22.1',
   'M.1.9',
   'C.1.9.0',
   'M.1.56',
   'C.1.56.2',
   'C.1.34.2',
   'M.1.0',
   'C.1.0.59',


In [10]:
knns2

{'33123312326462326462': {'33123312326462326462': 0.0,
  '32763276289893289893': 4.6714983,
  '32513251286862286862': 4.6865196,
  '33123312325262325262': 3.7112916,
  '32513251290462290462': 4.0787635,
  '32513251288062288062': 4.7495623,
  '33123312327662327662': 4.508333,
  '32763276273662273662': 4.8615317,
  '32153215334862334862': 4.9994154,
  '32763276274862274862': 4.6687303,
  '32513251271893271893': 4.868347,
  '32513251270062270062': 4.925503,
  '32513251289262289262': 4.851357,
  '33123312403293403293': 5.019895,
  '32153215336062336062': 4.6851144,
  '32763276310862310862': 4.6100664,
  '32513251274862274862': 4.857136,
  '33123312405662405662': 4.367864,
  '33123312380462380462': 4.7946477,
  '33123312324093324093': 3.8193817,
  '32513251271262271262': 3.8015993,
  '32763276292862292862': 4.729035,
  '32763276271893271893': 4.6991673,
  '32513251308462308462': 4.954049,
  '32513251272462272462': 3.7889712,
  '32763276307262307262': 4.9836183,
  '32513251270093270093': 4.7

In [15]:
evaluate_knn_per_query(search_result, df_new, knns2)

Evaluating k-NN performance on 9 checkpoints: [14344, 14344, 15907, 15907, 18491, 59808, 100493, 210505, 300462]
C.1.43.1
N. of knns found: 22 in 2 buckets.
C.1.43.1
N. of knns found: 22 in 3 buckets.
C.1.43.1
C.1.10.2
N. of knns found: 24 in 4 buckets.
C.1.43.1
C.1.10.2
N. of knns found: 24 in 5 buckets.
C.1.43.1
C.1.10.2
C.1.63.1
N. of knns found: 27 in 6 buckets.
C.1.43.1
C.1.10.2
C.1.63.1
C.1.34.1
C.1.25.8
N. of knns found: 30 in 94 buckets.
C.1.43.1
C.1.10.2
C.1.63.1
C.1.34.1
C.1.25.8
N. of knns found: 30 in 117 buckets.
C.1.43.1
C.1.10.2
C.1.63.1
C.1.34.1
C.1.25.8
N. of knns found: 30 in 331 buckets.
C.1.43.1
C.1.10.2
C.1.63.1
C.1.34.1
C.1.25.8
N. of knns found: 30 in 633 buckets.


[0.7333333333333333, 0.7333333333333333, 0.8, 0.8, 0.9, 1.0, 1.0, 1.0, 1.0]

In [19]:
queries[queries["object_id"] == "3392_3392-3060-93_3060_93"]

,L1_pred,L2_pred,L1,L2,object_id,0,1,2,3,4,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
6176,43,0,5,76,3392_3392-3060-93_3060_93,0.0,0.0,0.174825,0.0,0.0,...,0.0,0.0,0.05983,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df_result.head()

,L1_pred,L2_pred,L1,L2,object_id,0,1,2,3,4,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,55,3,1,78,3169_3169-1581-468_1581_468,0.0,0.0,0.346130,0.0,0.0,...,0.11011,0.0,0.0,0.0,0.0,0.265144,0.00000,0.0,0.0,0.038831
1,2,8,2,5,3137_3137-768-62_768_62,0.0,0.0,0.889276,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.000000,0.31331,0.0,0.0,0.000000
2,43,1,5,76,3417_3417-2286-93_2286_93,0.0,0.0,0.176439,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000
3,26,3,4,75,3195_3195-3804-62_3804_62,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000
4,58,1,1,17,3287_3287-189-139_189_139,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000


In [30]:
for i in range(len(queries)):
    if queries.iloc[i]["object_id"] == "3392_3392-3060-93_3060_93":
        print(i)
        break

15


In [ ]:
for i in range(query_count):
        
    search_result = li.search(df_new, queries.iloc[i]["object_id"], stop_cond_objects = object_checkpoints, debug = False)

    time_checkpoints = search_result['time_checkpoints']
    
    errors = []
    
    try:
        recall = evaluate_knn_per_query(search_result, df_new, knns2)
        for j in range(checkpoint_count):
            model_times[j] += time_checkpoints[j]
            model_recall[j] += recall[j]
            
    except IndexError:
        errors.append(i)

    
model_times = list(map(lambda x: x / query_count, model_times))
model_recall = list(map(lambda x: x / query_count, model_recall))

final_results[identifier] = {"model_times": model_times, "model_recall": model_recall}
print(final_results[identifier])

#with open(f"{self.destination}/{identifier}.csv", 'w') as f:
#    f.write("model_times,model_recall\n")
#    for i in range(checkpoint_count):
#        f.write("%f,%f\n"%(model_times[i], model_recall[i]))